In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy, qiskit
import circuit_cutter
import mlrecon_methods as ml
import json

import numpy as np
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, H_distance_dict, total_counts
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [3]:
with open('LiH_auckland_nocut_orign.json') as f:
    all_list = json.load(f)

In [4]:
len(all_list)

125

In [5]:
unmiti_list = all_list[-25:]
len(unmiti_list)

25

In [6]:
nocut_list = all_list[0:-25]
len(nocut_list)

100

In [7]:
with open('LiH_auckland_check0.json') as f:
    miti_q0_list = json.load(f)
with open('LiH_auckland_check1.json') as f:
    miti_q1_list = json.load(f)
with open('LiH_auckland_check2.json') as f:
    miti_q2_list = json.load(f)
with open('LiH_auckland_check3.json') as f:
    miti_q3_list = json.load(f)

In [8]:
per_miti_results = []
for i in range(0, 4):
    temp_list = []
    for j in range(0, 25):
        filter_per_miti = norm_dict(filter_results(nocut_list[i * 25 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [9]:
per_miti_results[0]

[{'0000': 0.07446256101360474,
  '0001': 0.05867691349049745,
  '0010': 0.063558001869353,
  '0011': 0.05265344272510126,
  '0100': 0.07186623740783052,
  '0101': 0.057846089936649706,
  '0110': 0.06231176653858137,
  '0111': 0.05244573683663932,
  '1000': 0.07321632568283311,
  '1001': 0.059715442932807145,
  '1010': 0.06667359019628206,
  '1011': 0.05774223699241873,
  '1100': 0.07238550212898535,
  '1101': 0.05794994288088067,
  '1110': 0.06324644303666009,
  '1111': 0.05524976633087548},
 {'0000': 0.001971772519717725,
  '0001': 0.0024906600249066002,
  '0010': 0.13210875882108758,
  '0011': 0.11498547114985472,
  '0100': 0.00311332503113325,
  '0101': 0.0030095475300954753,
  '0110': 0.12816521378165213,
  '0111': 0.10938148609381486,
  '1000': 0.0038397675383976754,
  '1001': 0.0030095475300954753,
  '1010': 0.13740141137401413,
  '1011': 0.11809879618098797,
  '1100': 0.003424657534246575,
  '1101': 0.0033208800332088003,
  '1110': 0.12826899128268993,
  '1111': 0.10740971357409

Untimigated distribution

In [10]:
qubits = 4
# list of all possible measurement outcomes (bitstrings)
all_bits = [ "".join(bits) for bits in itertools.product(["0","1"], repeat = qubits) ]

In [11]:
def load_probs(miti_dist_list, bit_str):
    output_list = []
    for dist in miti_dist_list:
        try:
            miti_count = dist[bit_str]
        except:
            miti_count = 0
        output_list.append(miti_count)
    return output_list

In [12]:
def bit_weight(dist, index):
    #bitwise distribution
    weight_0 = 0
    weight_1 = 0
    for key in dist.keys():
        if key[len(key) - 1 - index] == '0':
            weight_0 += dist[key]
        elif key[len(key) - 1 - index] == '1':
            weight_1 += dist[key]
        else:
            print("Incorrect key value")
    return weight_0, weight_1

In [13]:
def update_dist(unmiti_dist, miti_dist, index):
    Ppost = {}
    w0, w1 = bit_weight(miti_dist, index)
    u_w0, u_w1 = bit_weight(unmiti_dist, index)
    if w0 == 0:
        w0 = 0.0000000000001
        w1 = 0.9999999999999
    if w1 == 0:
        w1 = 0.0000000000001
        w0 = 0.9999999999999
    if u_w0 == 0:
        u_w0 = 0.0000000000001
        u_w1 = 0.9999999999999
    if u_w1 == 0:
        u_w1 = 0.0000000000001
        u_w0 = 0.9999999999999
        
    for key in unmiti_dist.keys():
        if key[len(key) - 1 - index] == '0':
            Ppost[key] = unmiti_dist[key] / u_w0 * (w0)# / w1)
            #print(w0, w1, w0/w1, Ppost[key])
        elif key[len(key) - 1 - index] == '1':
            Ppost[key] = unmiti_dist[key] / u_w1 * (w1)# / w0)
            #print(w0, w1, w1/w0, Ppost[key])
        else:
            print("Incorrect key value")
    return Ppost

In [14]:
def combine_dist(orign_dist, dist_list):
    output_dist = {}
    for key in orign_dist:
        value = orign_dist[key]
        for dist in dist_list:
            value += dist[key]
        output_dist[key] = value
    return output_dist  

In [15]:
def bayesian_reconstruct(unmiti_dist, miti_dist_list, threshold = 0.0001):
    temp_dist = unmiti_dist.copy()
    h_dist = 1
    while h_dist > threshold:
        temp_dist_start = temp_dist.copy()
        ppost = [0] * len(miti_dist_list)
        for i in range(0, len(miti_dist_list)):
            ppost[i] = update_dist(temp_dist, miti_dist_list[i][0], miti_dist_list[i][1])
        temp_dist = combine_dist(temp_dist, ppost)
        temp_dist = norm_dict(temp_dist)
        h_dist = H_distance_dict(temp_dist, temp_dist_start)
        print("H-dist:", h_dist)
    return temp_dist

In [16]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [17]:
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [per_miti_results[0][idx],0],[per_miti_results[1][idx],1], [per_miti_results[2][idx],2], [per_miti_results[3][idx],3]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133750461
H-dist: 0.014269069679544524
H-dist: 0.011373068099988908
H-dist: 0.009075253552205967
H-dist: 0.007246766484835398
H-dist: 0.005789136872557935
H-dist: 0.0046258677810851485
H-dist: 0.0036968934403554935
H-dist: 0.002954726476908162
H-dist: 0.002361661558715868
H-dist: 0.0018876796700977485
H-dist: 0.0015088415364153424
H-dist: 0.0012060370141846926
H-dist: 0.0009640017314638849
H-dist: 0.0007705388613295319
H-dist: 0.0006159007296379526
H-dist: 0.0004922961933136089
H-dist: 0.00039349760980785377
H-dist: 0.00031452697889180386
H-dist: 0.00025140517666907657
H-dist: 0.00020095154065070958
H-dist: 0.0001606236718728896
H-dist: 0.00012838937504769014
H-dist: 0.00010262428721106382
H-dist: 8.203004212681011e-05
H-dist: 70.00357133748567
H-dist: 0.00991960758671482
H-dist: 0.00795412102993478
H-dist: 0.006377387706830626
H-dist: 0.005111643654889883
H-dist: 0.004095498235033596
H-dist: 0.0032799818457057667
H-dist: 0.0026257990188734903
H-dist: 0.0021013216050537

In [18]:
pauli_list = read_from_file('LiH_hamiltonian_1.5.txt')
pauli_commute = find_commute_groups(pauli_list)

In [19]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [20]:
final_expect_val

(-1.0584306861322097+0j)

In [21]:
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [miti_q0_list[idx],0],[miti_q1_list[idx],1], [miti_q2_list[idx],2], [miti_q3_list[idx],3]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133752331
H-dist: 0.017484794466551476
H-dist: 0.013869538510378367
H-dist: 0.011035458998125922
H-dist: 0.008797181442044015
H-dist: 0.0070212290246465334
H-dist: 0.005607994945881222
H-dist: 0.004481332696434199
H-dist: 0.0035820921277806327
H-dist: 0.0028638430002696324
H-dist: 0.0022898924162659917
H-dist: 0.0018311166317667915
H-dist: 0.0014643356111143074
H-dist: 0.0011710675827981637
H-dist: 0.0009365604098685771
H-dist: 0.0007490307215777365
H-dist: 0.0005990623455259223
H-dist: 0.00047912868998102094
H-dist: 0.000383212521118278
H-dist: 0.0003065027616817095
H-dist: 0.0002451524516481744
H-dist: 0.00019608540506728565
H-dist: 0.000156841702389224
H-dist: 0.00012545418646691796
H-dist: 0.00010034972574322563
H-dist: 8.027027042434985e-05
H-dist: 70.00357133749326
H-dist: 0.012358959172967659
H-dist: 0.010812304257601196
H-dist: 0.009497197815153634
H-dist: 0.008370003918410761
H-dist: 0.007397077471771623
H-dist: 0.0065521911931704065
H-dist: 0.00581467253675723

In [22]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [23]:
final_expect_val

(-1.0805725613253052+0j)

In [24]:
#no bayesian recombine:

In [25]:
miti_dist_list = [miti_q0_list, miti_q1_list, miti_q2_list, miti_q3_list]
total = 0
for idx in range(0, 4):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(miti_dist_list[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-1.054895041713911+0j)
expectation value for mitigating qubit1:(-1.0648051426111258+0j)
expectation value for mitigating qubit2:(-1.0517798145636696+0j)
expectation value for mitigating qubit3:(-1.0574188580910056+0j)


In [26]:
#no cutting
total = 0
for idx in range(0, 4):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-1.0370390814212622+0j)
expectation value for mitigating qubit1:(-1.037039534719149+0j)
expectation value for mitigating qubit2:(-0.9351584043522796+0j)
expectation value for mitigating qubit3:(-1.0091429498937454+0j)


In [27]:
#untimigated result:
unmiti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        unmiti_expect_val += coeff * evaluation(unmiti_list[i], shots = total_counts(unmiti_list[i]), Pauli = Pauli_tuple[0])

In [28]:
unmiti_expect_val

(-1.0514831765722086+0j)